In [1]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
from livenessnet import LivenessNet
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import os

In [2]:
# construct the argument parser and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-d", "--dataset", required=True,
#     help="path to input dataset")
# ap.add_argument("-m", "--model", type=str, required=True,
#     help="path to trained model")
# ap.add_argument("-l", "--le", type=str, required=True,
#     help="path to label encoder")
# ap.add_argument("-p", "--plot", type=str, default="plot.png",
#     help="path to output loss/accuracy plot")
# args = vars(ap.parse_args())

In [2]:
args = {
    "dataset":"data4/",
    "model":"model4",
    "le":"le4",
    "plot":"plot4"
}

In [3]:
# initialize the initial learning rate, batch size, and number of
# epochs to train for
INIT_LR = 1e-4
BS = 8
EPOCHS = 50

In [4]:

# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
imagePaths = list(paths.list_images(args["dataset"]))

[INFO] loading images...


In [15]:
#print(imagePaths)

In [5]:
# loop over all image paths
data = []
labels = []
for imagePath in imagePaths:
    # extract the class label from the filename, load the image and
    # resize it to be a fixed 32x32 pixels, ignoring aspect ratio
    label = imagePath.split(os.path.sep)[-2]
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (32, 32))
    # update the data and labels lists, respectively
    data.append(image)
    labels.append(label)
# convert the data into a NumPy array, then preprocess it by scaling
# all pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0

In [6]:
# encode the labels (which are currently strings) as integers and then
# one-hot encode them
le = LabelEncoder()

In [7]:
# print(labels)
labels = le.fit_transform(labels)
# print(labels)
labels = to_categorical(labels, 2)
# print(labels)

In [8]:

# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
    test_size=0.25, random_state=42)

# construct the training image generator for data augmentation
aug = ImageDataGenerator(
    rotation_range=20, 
    zoom_range=0.15,
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    shear_range=0.25,
    horizontal_flip=True, fill_mode="nearest")
# initialize the optimizer and model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model = LivenessNet.build(width=32, height=32, depth=3,
    classes=len(le.classes_))
model.compile(loss="binary_crossentropy", optimizer=opt,
    metrics=["accuracy"])
# train the network
print("[INFO] training network for {} epochs...".format(EPOCHS))
H = model.fit(x=aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS)

# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(x=testX, batch_size=BS)
print(classification_report(testY.argmax(axis=1),
    predictions.argmax(axis=1), target_names=le.classes_))
# save the network to disk
print("[INFO] serializing network to '{}'...".format(args["model"]))
model.save(args["model"], save_format="h5")
# save the label encoder to disk
f = open(args["le"], "wb")
f.write(pickle.dumps(le))
f.close()
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, EPOCHS), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, EPOCHS), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, EPOCHS), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, EPOCHS), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(args["plot"])


[INFO] compiling model...
[INFO] training network for 50 epochs...
Train for 62 steps, validate on 166 samples
Epoch 1/50
62/62 [==============================] - 25s 406ms/step - loss: 1.1799 - accuracy: 0.5000 - val_loss: 0.6883 - val_accuracy: 0.5422
Epoch 2/50
62/62 [==============================] - 3s 55ms/step - loss: 0.9998 - accuracy: 0.5625 - val_loss: 0.6629 - val_accuracy: 0.6386
Epoch 3/50
62/62 [==============================] - 4s 58ms/step - loss: 0.8010 - accuracy: 0.6331 - val_loss: 0.6597 - val_accuracy: 0.6325
Epoch 4/50
62/62 [==============================] - 4s 57ms/step - loss: 0.9249 - accuracy: 0.6331 - val_loss: 0.6489 - val_accuracy: 0.6687
Epoch 5/50
62/62 [==============================] - 4s 60ms/step - loss: 0.9019 - accuracy: 0.6351 - val_loss: 0.6534 - val_accuracy: 0.6386
Epoch 6/50
62/62 [==============================] - 4s 57ms/step - loss: 0.8355 - accuracy: 0.6331 - val_loss: 0.5968 - val_accuracy: 0.7048
Epoch 7/50
62/62 [=======================